In [8]:
import warnings
warnings.filterwarnings('ignore')

In [30]:
import pandas as pd

coin = 'USDC'

path = '/Users/wanjeans/github project/Funding_rate_Aave/'

# Load the uploaded CSV files
rates_data_path = path + 'Nasim/rates_data.csv'
dai_price_data_path = path + 'Nikan/prices/' + coin + 'USDT.csv'

# Read the CSV files into DataFrames
df_rates = pd.read_csv(rates_data_path)
df_dai_price = pd.read_csv(dai_price_data_path)

# Filter the rates data to only include rows for "DAI"
df_rates_dai = df_rates[df_rates['Symbol'] == coin]

print(df_dai_price.columns)

print((df_rates_dai==0).sum())

# Calculate the rate changes (difference from previous row)
borrowrate_columns = ["liquidityRate_avg", "variableBorrowRate_avg", "utilizationRate_avg", "stableBorrowRate_avg"]
df_rates_dai_change = df_rates_dai[borrowrate_columns].diff()



# Add the calculated changes to the rates DataFrame
df_rates_dai["liquidityRate_change"] = df_rates_dai_change["liquidityRate_avg"]
df_rates_dai["variableBorrowRate_change"] = df_rates_dai_change["variableBorrowRate_avg"]
df_rates_dai["utilizationRate_change"] = df_rates_dai_change["utilizationRate_avg"]
df_rates_dai["stableBorrowRate_change"] = df_rates_dai_change["stableBorrowRate_avg"]

# Calculate the return rate for the price data
df_dai_price["return_rate"] = df_dai_price["close"].pct_change(periods=6)

# Merge the rates and price data on timestamp
df_combined = pd.merge(df_rates_dai, df_dai_price, left_on="Timestamp", right_on="timestamp")

# Calculate the correlation between rate changes and return rate
rate_change_columns = ["close","quote_asset_volume","liquidityRate_avg", "variableBorrowRate_avg", "utilizationRate_avg", "stableBorrowRate_avg", 
                       "liquidityRate_change", "variableBorrowRate_change", "utilizationRate_change", "stableBorrowRate_change"]
correlation = df_combined[["return_rate"] + rate_change_columns].dropna()
correlation = correlation.corr()


Index(['Unnamed: 0', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'close_time', 'quote_asset_volume', 'number_of_trades',
       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume',
       'ignore'],
      dtype='object')
Symbol                    0
Timestamp                 0
liquidityRate_avg         0
variableBorrowRate_avg    0
utilizationRate_avg       0
stableBorrowRate_avg      0
dtype: int64


In [31]:
correlation

,return_rate,close,quote_asset_volume,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,stableBorrowRate_avg,liquidityRate_change,variableBorrowRate_change,utilizationRate_change,stableBorrowRate_change
return_rate,1.000000,0.366535,0.110978,0.001136,0.000533,-0.001745,-0.000237,0.000545,-0.000196,-0.000154,-0.000154
close,0.366535,1.000000,0.081187,-0.003881,0.020438,0.013676,0.014389,-0.002779,-0.002474,-0.000871,-0.000693
quote_asset_volume,0.110978,0.081187,1.000000,0.151571,0.313032,0.101412,0.189440,0.006549,0.004773,0.009398,0.018232
liquidityRate_avg,0.001136,-0.003881,0.151571,1.000000,0.890938,0.094313,0.877503,0.267099,0.242756,-0.004120,0.178815
variableBorrowRate_avg,0.000533,0.020438,0.313032,0.890938,1.000000,0.188470,0.904643,0.226548,0.255544,-0.007602,0.174442
utilizationRate_avg,-0.001745,0.013676,0.101412,0.094313,0.188470,1.000000,0.120431,0.034780,0.052153,0.507347,0.049357
stableBorrowRate_avg,-0.000237,0.014389,0.189440,0.877503,0.904643,0.120431,1.000000,0.154765,0.159159,-0.014498,0.236639
liquidityRate_change,0.000545,-0.002779,0.006549,0.267099,0.226548,0.034780,0.154765,1.000000,0.901031,0.030827,0.681368
variableBorrowRate_change,-0.000196,-0.002474,0.004773,0.242756,0.255544,0.052153,0.159159,0.901031,1.000000,0.044872,0.686803
utilizationRate_change,-0.000154,-0.000871,0.009398,-0.004120,-0.007602,0.507347,-0.014498,0.030827,0.044872,1.000000,0.025645


In [11]:
unique_symbols = df_rates['Symbol'].unique()
unique_symbols

array(['DAI', 'TUSD', 'USDC', 'USDT', 'SUSD', 'BAT', 'LINK', 'KNC', 'MKR',
       'MANA', 'ZRX', 'SNX', 'WBTC', 'BUSD', 'ENJ', 'REN', 'YFI', 'AAVE',
       'UNI', 'AUSDT', 'AWBTC', 'AWETH', 'AYFI', 'AZRX', 'AUNI', 'AAAVE',
       'ABAT', 'ABUSD', 'ADAI', 'AENJ', 'AKNC', 'ALINK', 'AMANA', 'AMKR',
       'AREN', 'ASNX', 'ASUSD', 'ATUSD', 'AUSDC', 'ACRV', 'AGUSD', 'ABAL',
       'AXSUSHI', 'ARENFIL', 'ARAI', 'AAMPL', 'AUSDP', 'ADPI', 'AFRAX',
       'AFEI', 'ASTETH', 'AENS', 'AUST', 'ACVX', 'A1INCH', 'ALUSD',
       'AAMMWETH', 'AAMMDAI', 'AAMMUSDC', 'AAMMUSDT', 'AAMMWBTC', 'WETH',
       'LUSD', 'CRV', 'BAL', 'ENS', '1INCH', 'FRAX'], dtype=object)

In [12]:
len(unique_symbols)

68

In [13]:
import pandas as pd
import numpy as np

def cal_corr(coin):
    path = 'E:/Funding_rate_Aave/'

    # Load the uploaded CSV files
    rates_data_path = path + 'Nasim/rates_data.csv'
    dai_price_data_path = path + 'Nikan/prices/' + coin + 'USDT.csv'

    try:
        # Read the CSV files into DataFrames
        df_rates = pd.read_csv(rates_data_path)
        df_dai_price = pd.read_csv(dai_price_data_path)
    except FileNotFoundError:
        # Return a DataFrame with NaN values if the file is not found
        rate_change_columns = ["liquidityRate_avg", "variableBorrowRate_avg", "utilizationRate_avg", "stableBorrowRate_avg", 
                            "liquidityRate_change", "variableBorrowRate_change", "utilizationRate_change", "stableBorrowRate_change"]
        return_rate_column = ["return_rate"]
        all_columns = return_rate_column + rate_change_columns
        return pd.DataFrame(np.nan, index=[0], columns=all_columns).corr()

    # Filter the rates data to only include rows for "DAI"
    df_rates_dai = df_rates[df_rates['Symbol'] == coin]

    # Calculate the rate changes (difference from previous row)
    borrowrate_columns = ["liquidityRate_avg", "variableBorrowRate_avg", "utilizationRate_avg", "stableBorrowRate_avg"]
    df_rates_dai_change = df_rates_dai[borrowrate_columns].diff()

    # Add the calculated changes to the rates DataFrame
    df_rates_dai["liquidityRate_change"] = df_rates_dai_change["liquidityRate_avg"]
    df_rates_dai["variableBorrowRate_change"] = df_rates_dai_change["variableBorrowRate_avg"]
    df_rates_dai["utilizationRate_change"] = df_rates_dai_change["utilizationRate_avg"]
    df_rates_dai["stableBorrowRate_change"] = df_rates_dai_change["stableBorrowRate_avg"]

    # Calculate the return rate for the price data
    df_dai_price["return_rate"] = df_dai_price["close"].pct_change(periods=6)

    # Merge the rates and price data on timestamp
    df_combined = pd.merge(df_rates_dai, df_dai_price, left_on="Timestamp", right_on="timestamp")

    # Calculate the correlation between rate changes and return rate
    rate_change_columns = ["liquidityRate_avg", "variableBorrowRate_avg", "utilizationRate_avg", "stableBorrowRate_avg", 
                        "liquidityRate_change", "variableBorrowRate_change", "utilizationRate_change", "stableBorrowRate_change"]
    correlation = df_combined[["return_rate"] + rate_change_columns].dropna()
    correlation = correlation.corr()
    
    return correlation

In [14]:
import numpy as np
np.size(cal_corr('BTC'))

81

In [15]:
corr_result = np.zeros((len(unique_symbols),9,9))
np.size(corr_result)

5508

In [16]:
for i in range(0,len(unique_symbols)):
    corr_result[i] = cal_corr(unique_symbols[i])

In [19]:
import seaborn as sns
import matplotlib.pyplot as plt

cumulative_data = correlation.fillna(0)
valid_data_count = 0

for i in range(0, len(unique_symbols)):
    data = cal_corr(unique_symbols[i])
    if not data.isna().any().any():
        cumulative_data += data  # 累加每次的 9x9 DataFrame
        valid_data_count += 1    # 记录有效数据的次数
    if pd.notna(data['return_rate']['return_rate']):
        plt.figure(figsize=(10, 8))

        # Create a heatmap using seaborn to visualize the correlation matrix
        sns.heatmap(data, annot=True, cmap='coolwarm', center=0, linewidths=0.5, fmt=".2f")

        # Add title and labels
        plt.title('Correlation Matrix Heatmap with Coin: ' + unique_symbols[i])
        plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
        plt.yticks(rotation=0)  # Keep y-axis labels horizontal
        
        plt.show()
        

# 检查是否有有效数据
if valid_data_count > 0:
    # 计算 data 的平均值
    average_data = cumulative_data / valid_data_count

    # 显示平均 heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(average_data, annot=True, cmap='coolwarm', center=0, linewidths=0.5, fmt=".2f")

    # 添加标题和标签
    plt.title('Average Correlation Matrix Heatmap Across Symbols')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)

    plt.show()